1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


* __hit_rate@k__ - был ли хотя бы 1 релевантный товар среди топ-k рекомендованных
* __precision@k__ - какой % рекомендованных товаров юзер купил среди k рекомендованных

    hit_rate@k дает минимум информации: купил что-либо клиент или нет. В отличие от неё precision@k дает больше информации для понимания процесса: на сколько хорошо сработала RS.
    
    
Пример №1.
Сайт канцтоваров.
Если по запросу "Пенал" система подобрала в качестве рекомендаций товары типа: ручка, карандаш, ластик - то, precision, в теории, должна дать хороший % в случае успеха.

Пример №2.
Для оценки банера на главной странице сайта, я бы применил метрику hit_rate просто потому, что априри неизвестно, какой товар нужен потребителю. Соответственно, в случае успеха hit_rate покажет "1". 

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



$log_2$ в знаменателе применяется для того, чтобы в топе оказались более релевантные ответы. Вместо $log_2$ можно выбрать любую медленно возрастающую функцию, например $\sqrt{x}$. Cамо собой в метрике DCG также применяется $log_2$.
Логарифм применяется в функции ошибок $logloss$.

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

In [ ]:
# Корреляция Спирмена - Корреляция реального и прогнозируемого рангов рекомендаций
from recsys.evaluation.ranking import SpearmanRho

TEST_RANKING = [('classical', 25.0), ('piano', 75.0), ('baroque', 50.0), ('instrumental', 25.0)]
GT_RANKING = [('classical', 50.0), ('piano', 100.0), ('baroque', 25.0), ('instrumental', 25.0)]
spearman = SpearmanRho()
spearman.load(GT_RANKING, TEST_RANKING)
spearman.compute() #returns 0.5

In [ ]:
# Кендалл-t. Сравнивает рекомендуемый список со списком элементов, предпочитаемых пользователем.
from recsys.evaluation.ranking import KendallTau

TEST_RANKING = [('classical', 25.0), ('piano', 75.0), ('baroque', 50.0), ('instrumental', 25.0)]
GT_RANKING = [('classical', 50.0), ('piano', 100.0), ('baroque', 25.0), ('instrumental', 25.0)]
kendall = KendallTau()
kendall.load(GT_RANKING, TEST_RANKING)
kendall.compute() #returns 0.4

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 (expected reciprocal rank)  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

In [1]:
import pandas as pd
import numpy as np

In [2]:
boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  

In [3]:
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

precision_at_k(recommended, boughted, 8)

0.5

In [4]:
def recall_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

recall_at_k(recommended, boughted, 8)

0.6666666666666666

In [5]:
def ap_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

ap_k(recommended, boughted, 8)

0.16

In [6]:
bought_list = np.array(boughted)
recommended_list = np.array(recommended)
k = 8

recommended_list = recommended_list[:k]

r = np.isin(bought_list, recommended_list)

def dcg_at_k(r, method = 1):
    
    r = np.asfarray(r)[:k]
    
    if method == 0:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    elif method == 1:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    else:
        raise ValueError('method must be 0 or 1.')
        
def ndcg_at_k(r):   
    dcg_max = dcg_at_k(sorted(r, reverse=True))
    if not dcg_max:
        return 0.
    return dcg_at_k(r) / dcg_max

ndcg_at_k(r)

0.6533933590057491

In [7]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks 

reciprocal_rank(recommended, boughted)

1.45